In [39]:
import requests
from requests.exceptions import HTTPError
import pandas as pd
import datetime
import matplotlib
import time

In [40]:
filter = '4169'
region = 'DE-LU'
resolution = 'quarterhour'

# Abfragen aller verfügbaren Timestamps für die Zone
timestamp_url = f'https://www.smard.de/app/chart_data/{filter}/{region}/index_{resolution}.json'

available_timestamps = requests.get(timestamp_url).json()["timestamps"]

unix_now = int(time.time() * 1000)
unix_start_date = 1640995200000
unix_end_date = 1767139200000

# Nach den Start und Enddatum (dazu nur bis jetzt!) filtern
filtered_timestamps = [ts for ts in available_timestamps if unix_start_date <= ts <= unix_end_date and ts <= unix_now]

responses_list = []

# Das Ganze in der response_list speichern
for filtered_timestamp in filtered_timestamps:
    request_url = f'https://www.smard.de/app/chart_data/{filter}/{region}/{filter}_{region}_{resolution}_{filtered_timestamp}.json'
    response = requests.get(request_url).json()["series"]
    responses_list.append(response)

# Die Liste in ein Dataframe überführen
df = pd.concat([pd.DataFrame(resp) for resp in responses_list], ignore_index=True)

In [41]:
# Dataframe anschauen bzgl. Timestamp (UNIX-Format)
df

,0,1
0,1641164400000,0.31
1,1641165300000,0.31
2,1641166200000,0.31
3,1641167100000,0.31
4,1641168000000,-0.01
...,...,...
140443,1767563100000,90.68
140444,1767564000000,97.61
140445,1767564900000,89.73
140446,1767565800000,86.33


In [43]:
# Datetime anpassen
df[0] = pd.to_datetime(df[0], unit='ms')

In [46]:
# Schauen, ob Werte korrekt formatiert sind
df.head(20)

,0,1
0,2022-01-02 23:00:00,0.31
1,2022-01-02 23:15:00,0.31
2,2022-01-02 23:30:00,0.31
3,2022-01-02 23:45:00,0.31
4,2022-01-03 00:00:00,-0.01
5,2022-01-03 00:15:00,-0.01
6,2022-01-03 00:30:00,-0.01
7,2022-01-03 00:45:00,-0.01
8,2022-01-03 01:00:00,-0.07
9,2022-01-03 01:15:00,-0.07


In [47]:
df.tail(20)

,0,1
140428,2026-01-04 18:00:00,123.50
140429,2026-01-04 18:15:00,120.92
140430,2026-01-04 18:30:00,119.38
140431,2026-01-04 18:45:00,115.06
140432,2026-01-04 19:00:00,121.30
140433,2026-01-04 19:15:00,115.33
140434,2026-01-04 19:30:00,110.00
140435,2026-01-04 19:45:00,103.17
140436,2026-01-04 20:00:00,106.87
140437,2026-01-04 20:15:00,101.72


In [52]:
# Spalten umbenennen
df.rename(columns={0: 'timestamp'}, inplace=True)
df.rename(columns={1: 'price'}, inplace=True)

In [53]:
df

,timestamp,price
0,2022-01-02 23:00:00,0.31
1,2022-01-02 23:15:00,0.31
2,2022-01-02 23:30:00,0.31
3,2022-01-02 23:45:00,0.31
4,2022-01-03 00:00:00,-0.01
...,...,...
140443,2026-01-04 21:45:00,90.68
140444,2026-01-04 22:00:00,97.61
140445,2026-01-04 22:15:00,89.73
140446,2026-01-04 22:30:00,86.33


In [65]:
df.to_parquet('../data/raw/energyprice.parquet', engine='fastparquet', compression='snappy')
print('Parquet file saved successfully')

Parquet file saved successfully


In [68]:
# Kurzen Überblick über die Daten
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 140448 entries, 0 to 140447
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  140448 non-null  datetime64[ms]
 1   price      140448 non-null  float64       
dtypes: datetime64[ms](1), float64(1)
memory usage: 2.1 MB


In [69]:
# Wie viele Spalten sind leer?
df.isna().sum()

timestamp    0
price        0
dtype: int64